# Sekrety systemu typów w języku Julia

### 2019-01-23

### [Bogumił Kamiński](https://github.com/bkamins)

Podstawowe reguły:
* każda wartość w języku Julia jest obiektem
* każda wartość w momencie wykonania programu ma typ
* zmienne są dowiązane (binding) do wartości, zmienna jako taka nie ma typu

Jeśli mamy typ `T` i może istnieć wartość, która jest tego typu to taki typ nazywamy konkretnym (concrete)

In [1]:
isconcretetype(Float64)

true

In [2]:
isconcretetype(AbstractString)

false

Większość typów, które nie są konkretne jest abstrakcyjne. Typ abstrakcyjny może być nadtypem innych typów.

In [3]:
isabstracttype(Float64)

false

In [4]:
isabstracttype(AbstractString)

true

Są wyjątki, ale w praktyce nie jest to istotne

In [5]:
isabstracttype(Tuple{Any}), isconcretetype(Tuple{Any})

(false, false)

In [6]:
subtypes(AbstractString)

4-element Array{Any,1}:
 String            
 SubString         
 SubstitutionString
 Test.GenericString

In [7]:
supertype(Float64)

AbstractFloat

In [8]:
supertype(AbstractString)

Any

In [9]:
supertype(Any)

Any

Można łatwo wypisać listę podtypów danego typu:

In [10]:
function subtype_tree(t::Type)
    st = subtypes(t)
    length(st) == 0 && return string(t)
    "$t: (" * join(subtype_tree.(subtypes(t)), ", ") * ")"
end

subtype_tree (generic function with 1 method)

In [11]:
subtype_tree(Integer)

"Integer: (Bool, Signed: (BigInt, Int128, Int16, Int32, Int64, Int8), Unsigned: (UInt128, UInt16, UInt32, UInt64, UInt8))"

Najprostsze typy konkretne to typy prymitywne (primitive) - reprezenujące sekwencję określonej liczby bitów w pewnej lokacji pamięci

In [12]:
isprimitivetype(Int64)

true

In [13]:
isprimitivetype(String)

false

Z punktu widzenia Julia wartości o typie prymitywnych nie są możliwe do zmiany (można jedynie zmienić przypisanie wartości do zmiennej).

In [14]:
x = 1

1

In [15]:
x = 10 # rebinding wartości do zmiennej

10

Typy konkretne mogą być też złożone: niezmienialne (immutable) lub zmienialne (mutable)

In [16]:
struct ImMut
    x::Int64
end

In [17]:
mutable struct Mut
    x::Int64
end

In [18]:
immut = ImMut(10)

ImMut(10)

In [19]:
immut.x

10

In [20]:
immut.x = 100 # błąd, niezmienialny

ErrorException: type ImMut is immutable

In [21]:
mut = Mut(10)

Mut(10)

In [22]:
mut.x

10

In [23]:
mut.x = 100

100

Z praktycznego punktu widzenia typy niemutowalne są szybsze (za wyjątkiem szczególnego przypadku bardzo dużych typów niemutowalnych)

Typy niemutowalne są przekazywane przez wartość, a mutowalne przez referencję.

In [24]:
f(x) = x.x = 1000

f (generic function with 1 method)

In [25]:
f(mut)

1000

In [26]:
mut

Mut(1000)

Są też typy niedeklarowane (unie) - nie posiadają nazwy, są głównie wykorzystywane w sygnaturach funkcji lub parametrach typów

In [27]:
Union{Float64, Missing}

Union{Missing, Float64}

Typy mogą posiadać parametry (tzw. parametric types)

In [28]:
Array{Int64, 1}

Array{Int64,1}

In [29]:
struct Tpl{T}
    x::T
    y::T
end

In [30]:
Tpl(1, 2)

Tpl{Int64}(1, 2)

In [31]:
Tpl("a", "b")

Tpl{String}("a", "b")

In [32]:
Tpl("a", 1) # typy argumentów nie zgadzają się

MethodError: MethodError: no method matching Tpl(::String, ::Int64)
Closest candidates are:
  Tpl(::T, !Matched::T) where T at In[29]:2

Typy mogą posiadać aliasy

In [33]:
Int

Int64

In [34]:
Vector

Array{T,1} where T

### Typy specjalne: Tuple i NamedTuple

`Tuple`: niemutowalna, indeksowalna; lista argumentów pozycyjnych funkcji

`NamedTuple`: niemutowalna, indeksowalna, z nazwami pól; technicznie: anonimowy `struct` ze zedefiniowaną funkcją `getindex`

In [35]:
f(x...) = typeof(x)

f (generic function with 2 methods)

In [36]:
f(1, 1.0, true)

Tuple{Int64,Float64,Bool}

In [37]:
(a=1, b=2, c=3)

(a = 1, b = 2, c = 3)

In [38]:
()

()

In [39]:
typeof(())

Tuple{}

In [40]:
(1,)

(1,)

In [41]:
typeof((1,))

Tuple{Int64}

In [42]:
NamedTuple()

NamedTuple()

In [43]:
(a=1,)

(a = 1,)

### Hierarchia typów

In [44]:
Int <: Integer

true

In [45]:
String <: AbstractString

true

Typy parametryczne mogą być inwariantne, kowariantne lub kontrawariantne

In [46]:
Vector{Int} <: Vector{Integer}

false

In [47]:
Vector{Int} <: Vector{<:Integer}

true

In [48]:
Vector{Integer} <: Vector{>:Int}

true

Wyjątkiem są krotki, które są zawsze kowariantne, a opcjonalnie kontrawariantne

In [49]:
Tuple{Int} <: Tuple{Integer}

true

In [50]:
Tuple{Int} <: Tuple{<:Integer}

true

In [51]:
Tuple{Int} <: Tuple{>:Integer}

true

In [52]:
Tuple{Integer} <: Tuple{Int}

false

In [53]:
Tuple{Integer} <: Tuple{<:Int}

false

In [54]:
Tuple{Integer} <: Tuple{>:Int}

true

Ma to znaczenie gdy specyfikujemy sygnatury metod

In [55]:
p(x::Vector{Integer}) = sum(x)

p (generic function with 1 method)

In [56]:
p([1,2,3])

MethodError: MethodError: no method matching p(::Array{Int64,1})
Closest candidates are:
  p(!Matched::Array{Integer,1}) at In[55]:1

In [57]:
p(Integer[1,2,3])

6

In [58]:
q(x::Vector{<:Integer}) = sum(x)

q (generic function with 1 method)

In [59]:
q([1,2,3])

6

In [60]:
r(x::Vector{>:Missing}) = count(ismissing, x)

r (generic function with 1 method)

In [61]:
r([1,2,3])

MethodError: MethodError: no method matching r(::Array{Int64,1})
Closest candidates are:
  r(!Matched::Array{#s1,1} where #s1>:Missing) at In[60]:1

In [62]:
r([1, missing, 3])

1

In [63]:
r(["a", missing, 1])

1

In [64]:
r(Any[1,2,3])

0

### Po co to wszystko jest potrzebne w praktyce

In [65]:
import Base: promote_rule, +, -, *, convert, show

abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Fahrenheit <: Temperature
    value::Float64
end

Celsius(t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
Fahrenheit(t::Celsius)  = Fahrenheit(t.value*9/5 + 32)

convert(::Type{Celsius}, t::Fahrenheit)  = Celsius(t)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t)

promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)
+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)
*(x::Number, y::T) where {T <: Temperature} = T(x * y.value)

const °C = Celsius(1)
const °F = Fahrenheit(1)

show(io::IO, t::Celsius) = print(io, t.value, "°C")
show(io::IO, t::Fahrenheit) = print(io, t.value, "°F")

show (generic function with 326 methods)

In [66]:
x = 1°C

1.0°C

In [67]:
y = 1°F

1.0°F

In [68]:
x+x

2.0°C

In [69]:
y+y

2.0°F

In [70]:
2x

2.0°C

In [71]:
7y+5x

-8.88888888888889°C

In [72]:
function our_pi(n, T)
    s = one(T)
    f = one(T)
    for i::T in 1:n
        f *= i / (2i+1)
        s += f
    end
    2s
end

our_pi (generic function with 1 method)

In [73]:
for T in [Float16, Float64, BigFloat]
    display(T)
    display([our_pi(2^n, T) for n in 2:9] .- big(π))
end

Float16

8-element Array{BigFloat,1}:
 -4.393640358979323846264338327950288419716939937510582097494459230781640628619803e-02
 -2.920778589793238462643383279502884197169399375105820974944592307816406286198029e-03
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04
 -9.676535897932384626433832795028841971693993751058209749445923078164062861980295e-04

Float64

8-element Array{BigFloat,1}:
 -4.317995517709517539637050332880069379799947750010582097494459230781640628619803e-02
 -2.122972943642620687196138639994458007746791953230820974944592307816406286198029e-03
 -6.257552732991451957385031566516208582973593855820974944592307816406286198029454e-06
 -7.004720171274999130895783071511373531260582097494459230781640628619802945362503e-11
 -1.010643099614860550061511927700156355820974944592307816406286198029453625031821e-15
 -1.010643099614860550061511927700156355820974944592307816406286198029453625031821e-15
 -1.010643099614860550061511927700156355820974944592307816406286198029453625031821e-15
 -1.010643099614860550061511927700156355820974944592307816406286198029453625031821e-15

BigFloat

8-element Array{BigFloat,1}:
 -4.317995517709482576423068486680447149875670096240740827653189389511799358779893e-02
 -2.122972943642004076114879103939885104758542258366351294298441073429877782059472e-03
 -6.257552731792070508295743788487240468397453131043771934107556099005139032410049e-06
 -7.004625201779280778523170847860187316653933480300725911129361546127247477203004e-11
 -1.176247251749589002199551915966763734093321056007856229508136919374480975152605e-20
 -4.555467456395812571863354028257417095625126183776878405020450217779608346301807e-40
  1.036340226611333355046362223536047948533920043732353766202844416420231016379491e-76
  1.036340226611333355046362223536047948533920043732353766202844416420231016379491e-76

In [74]:
our_pi(1000, BigFloat) - π

1.036340226611333355046362223536047948533920043732353766202844416420231016379491e-76

In [75]:
setprecision(1000) do
    our_pi(1000, BigFloat) - π
end

3.73305447401287551596035817889526867846836578548683209848685735918386764390310253781776130839152440943837995972129697049686195008541612957936608326881572302493764266455330060109598030394360732604440196318506045247296205005918373516322071308450166041524279351541770592447787925691464383688807065164177119e-301

In [76]:
mysum(a::AbstractArray) = sum(a)
mysum(a::AbstractArray{>:Missing}) = sum(skipmissing(a))

mysum (generic function with 2 methods)

In [77]:
x, y = [1,2,3], [1,2,3,missing]

([1, 2, 3], Union{Missing, Int64}[1, 2, 3, missing])

In [78]:
sum(x), sum(y), mysum(x), mysum(y)

(6, missing, 6, 6)

In [79]:
unwrapper(T::Type) = (println("no missing allowed"); T)
unwrapper(::Type{Union{A,Missing}}) where {A} = (println("missing allowed"); A)
unwrap(x::AbstractArray{T}) where {T>:Missing} = unwrapper(T)

unwrap (generic function with 1 method)

In [80]:
unwrapper(Any)

missing allowed


Any

In [81]:
unwrapper(Int64)

no missing allowed


Int64

In [82]:
unwrapper(Union{Int, Float64})

no missing allowed


Union{Float64, Int64}

In [83]:
unwrapper(Union{Int, Float64, Missing})

missing allowed


Union{Float64, Int64}

In [84]:
unwrap([1,2,3])

MethodError: MethodError: no method matching unwrap(::Array{Int64,1})
Closest candidates are:
  unwrap(!Matched::AbstractArray{T>:Missing,N} where N) where T>:Missing at In[79]:3

In [85]:
unwrap([1,2,missing])

missing allowed


Int64

In [86]:
unwrap(Any[1,2,3])

missing allowed


Any

In [87]:
unwrap(Real[1,2,3])

MethodError: MethodError: no method matching unwrap(::Array{Real,1})
Closest candidates are:
  unwrap(!Matched::AbstractArray{T>:Missing,N} where N) where T>:Missing at In[79]:3

### Najczęstszy błąd

In [88]:
f(x::AbstractArray{Integer}) = count(==(0), x)

f (generic function with 3 methods)

In [89]:
f([1,2,3])

ErrorException: type Array has no field x

In [90]:
f(Integer[1,2,3])

0

In [91]:
g(x::AbstractArray{<:Integer}) = count(==(0), x)

g (generic function with 1 method)

In [92]:
g([1,2,3])

0

In [93]:
g(Integer[1,2,3])

0